### Downloading data for the task, some additonal corpora and functions

Downloading all necessary libraries

In [7]:
import sys
from os import path

import numpy as np
import pandas as pd

from pandas import read_csv
from evaluate import evaluate

import gensim
import logging
from sklearn.cluster import AffinityPropagation, SpectralClustering, KMeans
import requests
from pymystem3 import Mystem

Downloading data for the task

In [1]:
!git clone https://github.com/nlpub/russe-wsi-kit.git

Cloning into 'russe-wsi-kit'...
remote: Enumerating objects: 148, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 148 (delta 4), reused 22 (delta 4), pack-reused 116
Receiving objects: 100% (148/148), 3.83 MiB | 8.87 MiB/s, done.
Resolving deltas: 100% (59/59), done.


In [ ]:
sys.path.append('/content/russe-wsi-kit/')

Test data for the wiki-wiki dataset is changed manually

Getting Natasha module

In [2]:
! pip install navec

In [21]:
!wget https://storage.yandexcloud.net/natasha-navec/packs/navec_hudlit_v1_12B_500K_300d_100q.tar


--2021-11-28 14:21:51--  https://storage.yandexcloud.net/natasha-navec/packs/navec_hudlit_v1_12B_500K_300d_100q.tar
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 53012480 (51M) [application/x-tar]
Saving to: ‘navec_hudlit_v1_12B_500K_300d_100q.tar.1’

navec_hudlit_v1_12B 100%[===================>]  50.56M  16.2MB/s    in 3.1s    

2021-11-28 14:21:55 (16.2 MB/s) - ‘navec_hudlit_v1_12B_500K_300d_100q.tar.1’ saved [53012480/53012480]



In [14]:
from navec import Navec
path = 'navec_hudlit_v1_12B_500K_300d_100q.tar'
navec = Navec.load(path)

Getting fasttext module

In [246]:
!wget http://vectors.nlpl.eu/repository/20/214.zip

--2021-11-28 17:50:05--  http://vectors.nlpl.eu/repository/20/214.zip
Resolving vectors.nlpl.eu (vectors.nlpl.eu)... 129.240.189.181
Connecting to vectors.nlpl.eu (vectors.nlpl.eu)|129.240.189.181|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1920218982 (1.8G) [application/zip]
Saving to: ‘214.zip’

214.zip             100%[===================>]   1.79G  24.9MB/s    in 75s     

2021-11-28 17:51:20 (24.4 MB/s) - ‘214.zip’ saved [1920218982/1920218982]



In [247]:
!unzip 214.zip -d ru_fasttext_model

Archive:  214.zip
  inflating: ru_fasttext_model/meta.json  
  inflating: ru_fasttext_model/model.model  
  inflating: ru_fasttext_model/model.model.vectors_ngrams.npy  
  inflating: ru_fasttext_model/model.model.vectors.npy  
  inflating: ru_fasttext_model/model.model.vectors_vocab.npy  
  inflating: ru_fasttext_model/README  


! Additionaly corporas were downloaded from https://rusvectores.org/en/models/ manually and saved at drive/My Drive/Corpora/

Corporas that I tried are 

1) ruscorpora_upos_skipgram_300_5_2018.vec.gz

2) taiga_upos_skipgram_300_2_2018.vec.gz

3) ruwikiruscorpora_upos_skipgram_300_2_2018.vec.gz

4) news_upos_cbow_600_2_2018.vec.gz

### Data preprocessing

Tokenization, lemmatization, splitting, lowering and position tagging (needed for the word2vec models taken from rusvectors website) is made with the help of the function 
tag_mystem.

Originally the functuion was taken from here and simplified/adapted https://github.com/akutuzov/webvectors/blob/master/preprocessing/rus_preprocessing_mystem.py


In [15]:
def tag_mystem(
    text="Текст нужно передать функции в виде строки!", mapping=None):
  
    m = Mystem()
    processed = m.analyze(text)
    tagged = []

    for w in processed:
      if 'analysis' in w and len(w['analysis']) > 0: 
        lemma = w["analysis"][0]["lex"].lower().strip()
        pos = w["analysis"][0]["gr"].split(",")[0]
        pos = pos.split("=")[0].strip()
        if mapping:
            if pos in mapping:
                pos = mapping[pos]  
            else:
                pos = "X"  #на случай, если попадется тэг, которого нет в маппинге
        tagged.append(lemma.lower() + "_" + pos)
    return tagged

Функция tag_mystem на выход выдает морфомлогические тэги в формате Mystem, чтобы преобразовать их в тэги UPoS, которые принимает бльшое число используемых мной корпусов, необходим следующий mapping

In [16]:
mapping_url = "https://raw.githubusercontent.com/akutuzov/universal-pos-tags/4653e8a9154e93fe2f417c7fdb7a357b7d6ce333/ru-rnc.map"

mystem2upos = {}
r = requests.get(mapping_url, stream=True)
for pair in r.text.split("\n"):
    pair = pair.split()
    if len(pair) > 1:
        mystem2upos[pair[0]] = pair[1]

Пример преобразования изначального датасета с помощью функции tag_mystem

In [30]:
train_raw = pd.read_csv("./russe-wsi-kit/data/main/wiki-wiki/train.csv", sep='\t')

In [19]:
#Transformation of the field 'Word'
df_part = pd.DataFrame()
a = train_raw['word'].unique()
for i in range(len(a)):
  df_part.loc[i, 'word'] = a[i]
  df_part.loc[i, 'word_new'] = a[i]

Code needed to run Mystem in Colab correctly


In [23]:
!wget http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
!tar -xvf mystem-3.0-linux3.1-64bit.tar.gz
!cp mystem /root/.local/bin/mystem

--2021-11-28 14:23:42--  http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
Resolving download.cdn.yandex.net (download.cdn.yandex.net)... 5.45.205.244, 5.45.205.243, 5.45.205.242, ...
Connecting to download.cdn.yandex.net (download.cdn.yandex.net)|5.45.205.244|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: http://cache-man01i.cdn.yandex.net/download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz [following]
--2021-11-28 14:23:43--  http://cache-man01i.cdn.yandex.net/download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
Resolving cache-man01i.cdn.yandex.net (cache-man01i.cdn.yandex.net)... 5.45.205.221, 2a02:6b8::3:221
Connecting to cache-man01i.cdn.yandex.net (cache-man01i.cdn.yandex.net)|5.45.205.221|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16457938 (16M) [application/octet-stream]
Saving to: ‘mystem-3.0-linux3.1-64bit.tar.gz’

mystem-3.0-linux3.1 100%[===================>]  15.70M  9

In [31]:
%%time
for i in range(len(df_part)):
  new = tag_mystem(df_part.word.loc[i], mapping=mystem2upos)
  df_part.word_new.loc[i] = new[0]

CPU times: user 26.7 ms, sys: 90.2 ms, total: 117 ms
Wall time: 3.77 s


In [32]:
new = pd.merge(train_raw, df_part, on='word')
new.drop('word',1, inplace= True)
new.rename(columns={"word_new": "word"}, inplace = True)

In [34]:
#Transformation of the 'Context' field
%%time
for i in range(len(new)):
  context = tag_mystem(new.context.loc[i], mapping=mystem2upos)
  new.context.loc[i] = ' '.join(context)

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


CPU times: user 2.95 s, sys: 9.72 s, total: 12.7 s
Wall time: 6min 58s


In [41]:
new.to_csv("./russe-wsi-kit/data/main/wiki-wiki/train_wiki-wiki_tagged.csv", index=False, sep='\t')

Actually, the time for the large datasets such as active-dict takes approximately 1.5 hour to run. So I ran it in other places and saved. The preprocessed datasets can be found in attached folder. Further I asssume that they are situated in the respective folders in Colab. 

Like for wiki-wiki dataset, it must be saved by the following path: /russe-wsi-kit/data/main/wiki-wiki/

The files are named for each datset in the following manner: train_wiki-wiki_tagged.csv, test_wiki-wiki_tagged.csv

For some corpora I need data in the format witoug morphology tags. So to transforms datasets above like train_wiki-wiki_tagged.csv I will use the transformation below

Wiki-wiki

In [468]:
df_tags = pd.read_csv("./russe-wsi-kit/data/main/wiki-wiki/train_wiki-wiki_tagged.csv", sep='\t')
for i in range(len(df_tags)):
    target = df_tags.word.loc[i].split('_')[0]
    df_tags.word.loc[i] = target
for i in range(len(df_tags)):
  target = df_tags.context.loc[i]
  words = target.split()
  words = [word.split('_')[0] for word in words]
  newcontext = ' '.join(words)
  df_tags.context.loc[i] = newcontext
df_tags.to_csv("./russe-wsi-kit/data/main/wiki-wiki/train_wiki-wiki_notags.csv", index=False, sep='\t')

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


Bts-rnc

In [469]:
df_tags = pd.read_csv("./russe-wsi-kit/data/main/bts-rnc/train_bts-rnc_tagged.csv", sep='\t')
for i in range(len(df_tags)):
    target = df_tags.word.loc[i].split('_')[0]
    df_tags.word.loc[i] = target
for i in range(len(df_tags)):
  target = df_tags.context.loc[i]
  words = target.split()
  words = [word.split('_')[0] for word in words]
  newcontext = ' '.join(words)
  df_tags.context.loc[i] = newcontext
df_notags.to_csv("./russe-wsi-kit/data/main/bts-rnc/train_bts-rnc_notags.csv", index=False, sep='\t')

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


Active-dict

In [470]:
df_tags3 = pd.read_csv("./russe-wsi-kit/data/main/active-dict/train_active-dict_tagged.csv", sep='\t')
for i in range(len(df_tags)):
    target = df_tags.word.loc[i].split('_')[0]
    df_tags.word.loc[i] = target
for i in range(len(df_tags)):
  target = df_tags.context.loc[i]
  words = target.split()
  words = [word.split('_')[0] for word in words]
  newcontext = ' '.join(words)
  df_tags.context.loc[i] = newcontext
df_notags.to_csv("./russe-wsi-kit/data/main/active-dict/train_active-dict_notags.csv", index=False, sep='\t')

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


Further I also save to russe-wsi-kit/data/main corresponding datasets and use them

### Methodology

1) After text preprocessing I firstly take the baseline solution of this contest from here https://github.com/akutuzov/russian_wsi (actually, it was quaite challenging for me to start it running)

The solution is as follows

- In each cluster of words (here I mean, for instance, all meanings of word 'замок') we take the vector embeddings of each word of the context with the function fingerprint() and take the mean of the sum of the vectors.

- The resulting vectors are then ran through two step-clustering by the function predict_clusters(): first, with Affinity Propagation to evaluate the amount of clusters and after that the eastimated amount of clusters is used in Spectral Clustering.

2) The upgrade of the baseline solution is 

- trying different corpora, including those without position tagging 
- trying different clustering hyperparametes + changing at the second step Spectral Clustering to KMeans Clustering

In [227]:
def fingerprint_pos(text, model):
    """
    :param text: list of words
    :param model: word2vec model in Gensim format
    :return: average vector of words in text
    """
    # Creating list of all words in the document, which are present in the model
    #words = [w for w in text if w in model]
    words = [w for w in text if w in model]
    lexicon = list(set(words))
    lw = len(lexicon)
    if lw < 1:
        print('Empty lexicon in', text, file=sys.stderr)
        return np.zeros(model.vector_size)
    vectors = np.zeros((lw, model.vector_size))  # Creating empty matrix of vectors for words
    for i in list(range(lw)):  # Iterate over words in the text
        word = lexicon[i]
        vectors[i, :] = model[word] 
    semantic_fingerprint = np.sum(vectors, axis=0)  # Computing sum of all vectors in the document
    semantic_fingerprint = np.divide(semantic_fingerprint, lw)  # Computing average vector
    return semantic_fingerprint

def fingerprint_nopos(text, model):
    """
    :param text: list of words
    :param model: word2vec model in Gensim format
    :return: average vector of words in text
    """
    words = [w for w in text if w in model.vocab]
    length = 300
    lexicon = list(set(words))
    lw = len(lexicon)
    if lw < 1:
        print('Empty lexicon in', text, file=sys.stderr)
        return np.zeros(300)
    vectors = np.zeros((len(words), 300))  # Creating empty matrix of vectors for words
    for i in range(len(words)):  # Iterate over words in the text
        word =  words[i]
        vectors[i, :] = model[word] 
    semantic_fingerprint = np.sum(vectors, axis=0)  # Computing sum of all vectors in the document
    semantic_fingerprint = np.divide(semantic_fingerprint, len(words))  # Computing average vector
    return semantic_fingerprint

In [641]:
def predict_clusters_pos(model, df, damping, preference, ninit):
  predicted = []
  goldsenses = []
  for query in df.word.unique():
      print('Now analyzing', query, '...', file=sys.stderr)
      subset = df[df.word == query]
      contexts = []
      matrix = np.empty((subset.shape[0], model.vector_size))
      counter = 0
      lengths = []
      for line in subset.iterrows():
          con = line[1].context
          identifier = line[1].context_id
          label = query + str(identifier)
          contexts.append(label)
          if type(con) == float:
              print('Empty context at', label, file=sys.stderr)
              fp = np.zeros(model.vector_size)
          else:
              bow = con.split()
              bow = [b for b in bow if b != query]
              fp = fingerprint_pos(bow, model)
              lengths.append(len(bow))
          matrix[counter, :] = fp
          counter += 1
      clustering = AffinityPropagation(damping=damping, preference = preference,
                                      random_state=None).fit(matrix)

      nclusters = len(clustering.cluster_centers_indices_)
      if nclusters < 1:
          print('Fallback to 1 cluster!', file=sys.stderr)
          nclusters = 1
      elif nclusters == len(contexts):
          print('Fallback to 4 clusters!', file=sys.stderr)
          nclusters = 4
      clustering = SpectralClustering(n_clusters=nclusters, n_init= ninit, 
                                    assign_labels='discretize', n_jobs=2).fit(matrix)
      #clustering = KMeans(n_clusters=nclusters, 
       #                  random_state=None).fit(matrix)

    # End two-stage clustering
      cur_predicted = clustering.labels_.tolist()
      predicted += cur_predicted
      print('Predicted clusters:', len(set(cur_predicted)), file=sys.stderr)

  df.predict_sense_id = predicted

  return df

def predict_clusters_nopos(model, df, damping, preference):
  predicted = []
  goldsenses = []
  for query in df.word.unique():
      print('Now analyzing', query, '...', file=sys.stderr)
      subset = df[df.word == query]
      contexts = []
      matrix = np.empty((subset.shape[0], 300))
      counter = 0
      lengths = []
      for line in subset.iterrows():
          con = line[1].context
          identifier = line[1].context_id
          label = query + str(identifier)
          contexts.append(label)
          if type(con) == float:
              print('Empty context at', label, file=sys.stderr)
              fp = np.zeros(model.vector_size)
          else:
              bow = con.split()
              bow = [b for b in bow if b != query]
              fp = fingerprint_nopos(bow, model)
              lengths.append(len(bow))
          matrix[counter, :] = fp
          counter += 1
      clustering = AffinityPropagation(damping=damping, preference = preference,
                                      random_state=None).fit(matrix)

      nclusters = len(clustering.cluster_centers_indices_)
      if nclusters < 1:
          print('Fallback to 1 cluster!', file=sys.stderr)
          nclusters = 1
      elif nclusters == len(contexts):
          print('Fallback to 4 clusters!', file=sys.stderr)
          nclusters = 4
      clustering = SpectralClustering(n_clusters=nclusters, n_init=25,
                                    assign_labels='discretize', n_jobs=2).fit(matrix)
      #clustering = KMeans(n_clusters=nclusters, 
      #                   random_state=None).fit(matrix)

    # End two-stage clustering
      cur_predicted = clustering.labels_.tolist()
      predicted += cur_predicted
      print('Predicted clusters:', len(set(cur_predicted)), file=sys.stderr)

  df.predict_sense_id = predicted

  return df

### Training phase

How to run the solution?

1) Choose corpora for train (if from block №1 uncomment one of the corporas and model, if from block №2 choose just one of the models)

2) Choose parameters for the first step of clustering (damping, preference)

3) Choose dataset with tags or with no tags 
- in case you use chose the model from block 1, then choose dataset with tags
- in case you use chose the model from block 2, then choose dataset with no tags

4) Choose function predict_clusters_pos() if you use corpora from block1, choose predict_clusters_pos() if you use corpora from block2

5) If you want to change the method of clustering at the second step, it should be done manually - in either function choose predict_clusters_pos() or in function choose predict_clusters_nopos() comment/uncomment rows with Spectral Clustering/KMeans

Great! You are beautiful! 


Step 1. Corpora

In [503]:
#1
modelfile = "drive/My Drive/Corpora/ruscorpora_upos_skipgram_300_5_2018.vec.gz"
#modelfile = "drive/My Drive/Corpora/taiga_upos_skipgram_300_2_2018.vec.gz"
#modelfile = "drive/My Drive/Corpora/ruwikiruscorpora_upos_skipgram_300_2_2018.vec.gz"
#modelfile = "drive/My Drive/Corpora/news_upos_cbow_600_2_2018.vec.gz"
model = gensim.models.KeyedVectors.load_word2vec_format(modelfile, binary=False)

#2
#model = gensim.models.KeyedVectors.load('ru_fasttext_model/model.model')
#model = navec

Step 2. Parameters to use

In [633]:
ninit = 25
damping = 0.8
preference = -0.5
#preference = -23

Step 3. Training dataset

Wiki-wiki

In [580]:
df = pd.read_csv("./russe-wsi-kit/data/main/wiki-wiki/train_wiki-wiki_tagged.csv", sep='\t')
#df = pd.read_csv("./russe-wsi-kit/data/main/wiki-wiki/train_wiki-wiki_notags.csv", sep='\t')

Bts-rnc

In [481]:
df = pd.read_csv("./russe-wsi-kit/data/main/bts-rnc/train_bts-rnc_tagged.csv", sep='\t')
#df = pd.read_csv("./russe-wsi-kit/data/main/bts-rnc/train_bts-rnc_notags.csv", sep='\t')

Active-dict

In [634]:
df = pd.read_csv("./russe-wsi-kit/data/main/active-dict/train_active-dict_tagged.csv", sep='\t')
#df = pd.read_csv("./russe-wsi-kit/data/main/active-dict/train_active-dict_notags.csv", sep='\t')

Step 4. Predict function

In [635]:
df_pred = predict_clusters_pos(model, df, damping, preference, ninit)
#df_pred = predict_clusters_nopos(model, df, damping, preference)

Now analyzing дар_NOUN ...
Predicted clusters: 3
Now analyzing двигатель_NOUN ...
Predicted clusters: 6
Now analyzing двойник_NOUN ...
Predicted clusters: 4
Now analyzing дворец_NOUN ...
Predicted clusters: 4
Now analyzing девятка_NOUN ...
Predicted clusters: 8
Now analyzing дедушка_NOUN ...
Empty lexicon in ['быть_VERB']
Predicted clusters: 1
Now analyzing дежурная_ADJ ...
Predicted clusters: 2
Now analyzing дежурный_NOUN ...
Predicted clusters: 2
Now analyzing декабрист_NOUN ...
Predicted clusters: 3
Now analyzing декрет_NOUN ...
Predicted clusters: 1
Now analyzing дело_NOUN ...
Empty lexicon in []
Empty lexicon in []
Empty lexicon in ['тут_ADV']
Empty lexicon in []
Empty lexicon in []
Empty lexicon in ['быть_VERB']
Empty lexicon in ['как_ADV']
Empty lexicon in ['быть_VERB']
Empty lexicon in []
Predicted clusters: 26
Now analyzing демобилизация_NOUN ...
Predicted clusters: 2
Now analyzing демократ_NOUN ...
Empty lexicon in []
Predicted clusters: 4
Now analyzing демонстрация_NOUN ...


### Evaluation

Wiki-wiki

In [582]:
train = pd.read_csv("./russe-wsi-kit/data/main/wiki-wiki/train.csv", sep='\t')
for i in range(len(train)):
  train.predict_sense_id[i] = df_pred.predict_sense_id[i]
train["predict_sense_id"] = train["predict_sense_id"].astype(int)
train.to_csv("./russe-wsi-kit/data/main/wiki-wiki/train_wiki-wiki_predicted.csv", sep='\t')
!python3 russe-wsi-kit/evaluate.py russe-wsi-kit/data/main/wiki-wiki/train_wiki-wiki_predicted.csv

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


word	ari	count
бор	0.000000	56
замок	0.880707	138
лук	0.963609	110
суда	0.562094	135
	0.691155	439


Bts-rnc

In [483]:
train = pd.read_csv("./russe-wsi-kit/data/main/bts-rnc/train.csv", sep='\t')
for i in range(len(train)):
  train.predict_sense_id[i] = df_pred.predict_sense_id[i]
train["predict_sense_id"] = train["predict_sense_id"].astype(int)
train.to_csv("./russe-wsi-kit/data/main/bts-rnc/train_bts-rnc_predicted.csv", sep='\t')
!python3 russe-wsi-kit/evaluate.py russe-wsi-kit/data/main/bts-rnc/train_bts-rnc_predicted.csv

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


word	ari	count
балка	-0.008102	119
вид	0.424842	77
винт	0.037193	123
горн	0.000000	51
губа	0.052632	137
жаба	0.025750	121
клетка	0.701126	150
крыло	0.059483	91
купюра	-0.004951	150
курица	0.177132	93
лавка	0.221781	149
лайка	0.040071	99
лев	0.000000	44
лира	0.000000	49
мина	0.000000	65
мишень	0.144932	121
обед	-0.004141	100
оклад	0.031907	146
опушка	0.006529	148
полис	-0.015060	142
пост	0.467878	144
поток	0.059190	136
проказа	0.196820	146
пропасть	0.118576	127
проспект	0.002546	139
пытка	-0.006056	143
рысь	0.605715	120
среда	0.323920	144
хвост	0.372095	121
штамп	0.240302	96
	0.153742	3491


Active-dict

In [636]:
train = pd.read_csv("./russe-wsi-kit/data/main/active-dict/train.csv", sep='\t')
for i in range(len(train)):
  train.predict_sense_id[i] = df.predict_sense_id[i]
train["predict_sense_id"] = train["predict_sense_id"].astype(int)
train.to_csv("./russe-wsi-kit/data/main/active-dict/train_active-dict_predicted.csv", sep='\t')
!python3 russe-wsi-kit/evaluate.py russe-wsi-kit/data/main/active-dict/train_active-dict_predicted.csv

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


word	ari	count
дар	0.058235	36
двигатель	0.300000	15
двойник	0.298016	25
дворец	0.446247	13
девятка	0.199738	47
дедушка	0.000000	9
дежурная	0.063830	12
дежурный	-0.022472	13
декабрист	0.633822	11
декрет	0.000000	12
дело	0.099770	130
демобилизация	0.271647	14
демократ	0.052883	18
демонстрация	0.093611	38
дерево	0.386364	21
держава	-0.068702	15
дерзость	0.071848	37
десятка	0.196124	36
десяток	0.148955	21
деятель	0.494700	14
диалог	0.157618	14
диаметр	0.298750	18
диплом	0.216687	25
директор	0.000000	11
диск	0.234134	63
дичь	0.117127	18
длина	0.163993	21
доброволец	0.380282	12
добыча	0.222019	35
доказательство	0.139439	24
доктор	0.561778	17
долгота	0.151020	13
доля	0.238265	45
дом	0.300436	38
дорога	0.283524	47
достижение	0.203636	22
древесина	-0.020067	16
дупло	0.309211	15
дура	0.435897	12
дух	0.077768	77
дым	0.394419	28
дымка	0.185304	18
дыхание	0.316065	56
дьявол	0.196182	22
евро	0.555556	8
езда	-0.116430	14
жаворонок	0.000000	11
жало	0.037500	11
жертва	0.241161	37
жестокость	-0.052670	

### Results


####Baseline

The results presented below are the baseline ones. The baseline parameters are: damping = 0.7, preference = -0.7 (first step clustering) for the first for corpora. For the ru_fasttexts baseline parameters are: damping = 0.7, preference = -10; for navec: damping = 0.7, preference = -23.

As you can see for the given solution and parameters for the bts-rnc dataset all corporas give pretty similar results. But for wiki-wiki and active-dict datasets they are much worser.

For the further optimizations I decided to stop at the first ruscorpora_upos_skipgram_300_5_2018.


| Corpora/model| Wiki-wiki | Bts-rnc | Active-dict |
| --- | --- | --- | --- |
| ruscorpora_upos_skipgram_300_5_2018| 0.628726 | 0.134935 | 0.182949 |
| taiga_upos_skipgram_300_2_2018| 0.536434 | 0.118117| 0.168383 |
| ruwikiruscorpora_upos_skipgram_300_2_2018| 0.510534| 0.112039 | 0.162747 |
| news_upos_cbow_600_2_2018| 0.565988 |0.105056 | 0.102181|
| ru_fasttext| 0.327494 |0.101594 | 0.157072|
| navec | 0.308518 | 0.101310 |0.139624|

####Best results

To obtain upgraded results not mentioning attempts of trying new corpora, there were tried different hyperparameters of the different combinations of Affinity Propagation, Kmeans ans Spectral Clustering. The best results are preseneted below in the table.

The best combinations were the ones of combination of Affinity Propagation at the first step 

The best params for Affinity Propagation are

- wiki-wiki: damping = 0.5, prefrence = -1
- bts-rnc: damping = 0.8, preference = -1.1
- active-dict: damping = 0.8, preference = -0.5, 

At the second step best models are KMeans for wiki-wiki and bts-rnc and Affinity Propagation with ninit = 25

| | Wiki-wiki | Bts-rnc | Active-dict |
| --- | --- | --- | --- |
| Baseline | 0.628726 | 0.134935 | 0.182949 |
| Upgraded | 0.691155 | 0.153742 | 0.206143 |

#### Discussion

Overall, the obtained results are great and there was some imporevement achevied throught the solution. 
In case there were no NLA course much more could have been done: I have chosen costly data preprocessing,
it would be great to remove surnames, numbers and play with the length of the context around target word.

Speaking about models, I have thought about trying ELMo vectors, smart parameter tuning via GridSearch,
another clustering techniques, and first place idea of this contest(finding similar words to the target words 
and measuring cosine similarity between context and difference bewteen target word and the most similar term to
the target).


###Predict 

In the final section I present the prediction made to the test files of the best models obtained in the train part. Actually, if there will be some problems with the submissions on the Codalab I would like these results to be assumed the final ones. It will also be possible to find them in the zip in files: "wiki-wiki_predicted.csv", "bts-rnc_predicted.csv", "active-dict_predicted.csv".


#### Wiki-wiki

In [471]:
modelfile = "drive/My Drive/Corpora/ruscorpora_upos_skipgram_300_5_2018.vec.gz"
model = gensim.models.KeyedVectors.load_word2vec_format(modelfile, binary=False)

In [639]:
damping = 0.5
preference = -1
ninit = 20

df = pd.read_csv("./russe-wsi-kit/data/main/wiki-wiki/test_wiki-wiki_tagged.csv", sep='\t')

df_pred = predict_clusters_pos(model, df, damping, preference, ninit)

test = pd.read_csv("./russe-wsi-kit/data/main/wiki-wiki/test.csv", sep='\t')
for i in range(len(test)):
  test.predict_sense_id[i] = df_pred.predict_sense_id[i]
test["predict_sense_id"] = test["predict_sense_id"].astype(int)
test.to_csv("./russe-wsi-kit/data/main/wiki-wiki/wiki-wiki_predicted.csv", sep='\t')


Now analyzing банк_NOUN ...
Predicted clusters: 3
Now analyzing белок_NOUN ...
Predicted clusters: 2
Now analyzing бит_NOUN ...
Predicted clusters: 2
Now analyzing горе_NOUN ...
Predicted clusters: 2
Now analyzing граната_NOUN ...
Predicted clusters: 1
Now analyzing граф_NOUN ...
Predicted clusters: 2
Now analyzing душа_NOUN ...
Predicted clusters: 1
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


Bts-rnc

In [640]:
damping = 0.8
preference = -1.1
ninit = 20

df = pd.read_csv("./russe-wsi-kit/data/main/bts-rnc/test_bts-rnc_tagged.csv", sep='\t')

df_pred = predict_clusters_pos(model, df, damping, preference, ninit)

test = pd.read_csv("./russe-wsi-kit/data/main/bts-rnc/test.csv", sep='\t')
for i in range(len(test)):
  test.predict_sense_id[i] = df_pred.predict_sense_id[i]
test["predict_sense_id"] = test["predict_sense_id"].astype(int)
test.to_csv("./russe-wsi-kit/data/main/bts-rnc/bts-rnc_predicted.csv", sep='\t')

Now analyzing акция_NOUN ...
Predicted clusters: 2
Now analyzing баба_NOUN ...
Predicted clusters: 2
Now analyzing байка_NOUN ...
Predicted clusters: 3
Now analyzing бум_INTJ ...
Predicted clusters: 1
Now analyzing бычок_NOUN ...
Predicted clusters: 2
Now analyzing вал_NOUN ...
Predicted clusters: 2
Now analyzing газ_NOUN ...
Predicted clusters: 2
Now analyzing гвоздик_NOUN ...
Predicted clusters: 2
Now analyzing гипербола_NOUN ...
Predicted clusters: 1
Now analyzing град_NOUN ...
Predicted clusters: 3
Now analyzing гусеница_NOUN ...
Predicted clusters: 2
Now analyzing дождь_NOUN ...
Predicted clusters: 2
Now analyzing домино_NOUN ...
Predicted clusters: 2
Now analyzing забой_NOUN ...
Predicted clusters: 1
Now analyzing икра_NOUN ...
Predicted clusters: 2
Now analyzing кабачок_NOUN ...
Predicted clusters: 3
Now analyzing капот_NOUN ...
Predicted clusters: 2
Now analyzing карьер_NOUN ...
Predicted clusters: 2
Now analyzing кличка_NOUN ...
Predicted clusters: 2
Now analyzing ключ_NOUN ..

Active-dict

In [642]:
damping = 0.8
preference = -0.5
ninit = 25

df = pd.read_csv("./russe-wsi-kit/data/main/active-dict/test_active-dict_tagged.csv", sep='\t')

df_pred = predict_clusters_pos(model, df, damping, preference, ninit)

test = pd.read_csv("./russe-wsi-kit/data/main/active-dict/test.csv", sep='\t')
for i in range(len(test)):
  test.predict_sense_id[i] = df_pred.predict_sense_id[i]
test["predict_sense_id"] = test["predict_sense_id"].astype(int)
test.to_csv("./russe-wsi-kit/data/main/active-dict/active-dict_predicted.csv", sep='\t')

Now analyzing давление_NOUN ...
Empty lexicon in ['у_ADP', 'он_PRON', 'всегда_ADV', 'на_ADP']
Predicted clusters: 3
Now analyzing дама_NOUN ...
Predicted clusters: 3
Now analyzing данные_NOUN ...
Predicted clusters: 6
Now analyzing дата_NOUN ...
Predicted clusters: 1
Now analyzing двойка_NOUN ...
Predicted clusters: 9
Now analyzing двор_NOUN ...
Predicted clusters: 2
Now analyzing дворник_NOUN ...
Predicted clusters: 1
Now analyzing девка_NOUN ...
Predicted clusters: 15
Now analyzing девочка_NOUN ...
Predicted clusters: 5
Now analyzing девушка_NOUN ...
Empty lexicon in ['у_ADP', 'ты_PRON', 'быть_VERB']
Predicted clusters: 5
Now analyzing девчонка_NOUN ...
Predicted clusters: 7
Now analyzing дед_NOUN ...
Predicted clusters: 5
Now analyzing дезертир_NOUN ...
Predicted clusters: 4
Now analyzing действие_NOUN ...
Empty lexicon in ['между_ADP']
Predicted clusters: 14
Now analyzing действительность_NOUN ...
Predicted clusters: 3
Now analyzing декларация_NOUN ...
Predicted clusters: 3
Now ana



---

